In [1]:
import pandas as pd
# Helpful for showing indexing information
pd.set_option('display.max_colwidth', 150)

import os
import pyterrier as pt

import platform
platform.uname()

uname_result(system='Linux', node='gl1000.arc-ts.umich.edu', release='4.18.0-305.45.1.el8_4.x86_64', version='#1 SMP Wed Apr 6 13:48:37 EDT 2022', machine='x86_64')

In [2]:
if not pt.started():
    pt.init()

PyTerrier 0.8.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [5]:
df_anime=pd.read_csv('data_processed/anime_with_review.csv')
df_anime=df_anime.rename(columns={'uid':'docno'})
df_anime.head(1)

,docno,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link,wiki_link,anime_name,wiki_intro,review
0,28891,Haikyuu!! Second Season,"Following their participation at the Inter-High, the Karasuno High School volleyball team attempts to refocus their efforts, aiming to conquer the...","['Comedy', 'Sports', 'Drama', 'School', 'Shounen']","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/76662.jpg,https://myanimelist.net/anime/28891/Haikyuu_Second_Season,http://en.wikipedia.org/wiki/Haiky%C5%AB!!,Haikyū!!,"Junior high school student, Shoyo Hinata, becomes obsessed with volleyball after catching a glimpse of Karasuno High School playing in Nationals o...","Overall: 10 \r\n \r\nPlot: 10 \r\n \r\nHaikyuu! is a sports genre anime, mainly tackling about Volleyball. In this kind of story, there's only one..."


In [6]:
df_anime['docno']=df_anime['docno'].astype(str)
df_anime['popularity']=df_anime['popularity'].astype(str)
df_anime['ranked']=df_anime['ranked'].astype(str)
df_anime['score']=df_anime['score'].astype(str)
df_anime['episodes']=df_anime['episodes'].astype(str)

In [3]:

pt_index_path = './pt_index'

if not os.path.exists(pt_index_path + "/data.properties"):

    indexer = pt.DFIndexer(pt_index_path,overwrite=True)
    index_ref = indexer.index(
    df_anime['synopsis'],df_anime['docno'],df_anime['title'],df_anime['popularity'],df_anime['ranked'],df_anime['score'],
    df_anime['episodes'],df_anime['wiki_intro'],df_anime['review']
)

else:
    # if you already have the index, create an IndexRef from the data in pt_index_path
    # that we can use to load using the IndexFactory
    # TODO
    index_ref = pt.IndexRef.of(pt_index_path+"/data.properties")
index = pt.IndexFactory.of(index_ref)

In [4]:
bm25_150 = pt.BatchRetrieve(index, wmodel="BM25")%150
bm25=pt.BatchRetrieve(index,wmodel='BM25')

In [5]:
query=pd.read_csv('data_processed/query_list.csv')
query.reset_index(inplace=True)
query.rename(columns={'Query':'query','index':'qid'},inplace=True)
train_query=query[query['Train/Test']=='Train'].drop(columns=['Train/Test'])
test_query=query[query['Train/Test']=='Test'].drop(columns=['Train/Test'])

In [6]:
df_query=train_query.append(test_query)
df_query=df_query.reset_index().drop(columns=['index'])

In [15]:
result=bm25_150(train_query)
result=result.merge(df_anime,on=['docno'],how='left')
result=result[['qid','docno','rank','score_x','query','title','synopsis','genre','wiki_intro','review']]

In [16]:
result.rename(columns={'score_x':'bm25_score'})
result.to_csv('system_rating/baseline_rating.csv',index=False)

In [17]:
result_test=bm25_150(test_query)
result_test=result_test.merge(df_anime,on=['docno'],how='left')
result_test=result_test[['qid','docno','rank','score_x','query','title','synopsis','genre','wiki_intro','review']]
result_test.rename(columns={'score_x':'bm25_score'})
result_test.to_csv('system_rating/baseline_rating_test.csv',index=False)

In [27]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
df_test=bm25("High school idol")


/home/zihuiliu/.local/lib/python3.9/site-packages/pyterrier/transformer.py:269: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  return self.transform(*args, **kwargs)


In [28]:
df_test

,qid,docid,docno,rank,score,query
0,1,2581,32526,0,15.116253,High school idol
1,1,1867,15051,1,14.667413,High school idol
2,1,2119,22189,2,13.289305,High school idol
3,1,229,24997,3,12.606818,High school idol
4,1,4499,10321,4,12.383107,High school idol
...,...,...,...,...,...,...
995,1,3407,3838,995,1.681246,High school idol
996,1,3832,1017,996,1.681246,High school idol
997,1,4398,395,997,1.681246,High school idol
998,1,672,17895,998,1.670446,High school idol


In [22]:
print(index.getCollectionStatistics().toString())

Number of documents: 4517
Number of terms: 20551
Number of postings: 214508
Number of fields: 0
Number of tokens: 255416
Field names: []
Positions:   false



## Evaluate baseline

### Naive random baseline

In [7]:
import random

def random_weighting(keyFreq, posting, entryStats, collStats):
    '''
    Computes the Pivoted Normalization score of this document for a given query
    
    :param keyFrequency(float): the weight of the term in the query, usually 1 except during PRF.
    :param posting(Posting): access to the information about the occurrence of the term in the current document (frequency, document length etc).
    :param entryStats(EntryStatistics): access to the information about the occurrence of the term in the whole index (document frequency, etc.).
    :param collStats(CollectionStatistics): access to the information about the index as a whole (number of documents, etc).
    :return: the Pivoted Normalization score of this document for the given query
    '''
    return random.random()
random_model=pt.BatchRetrieve(index,wmodel=random_weighting)

In [8]:


df_query2=df_query[df_query['qid']<10]
df_query2['qid']=df_query2['qid'].astype('str')
df_label2=pd.read_csv("labels.csv")
df_label2['qid']=df_label2['qid'].astype('str')
df_label2['docno']=df_label2['docno'].astype('str')
df_label2['label']=df_label2['label'].astype('int64')

/tmp/ipykernel_3764012/1103802641.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_query2['qid']=df_query2['qid'].astype('str')


In [9]:
df_label2[df_label2['label'].isna()]

,qid,docno,label


In [11]:
from pyterrier.measures import *


pt.Experiment(
    [bm25,bm25_150,random_model],
    df_query2,
    df_label2,
    eval_metrics=["map", "ndcg",nDCG@5,nDCG@10])

,name,map,ndcg,nDCG@5,nDCG@10
0,BR(BM25),0.641219,0.800433,0.646783,0.573949
1,"RankCutoff(BR(BM25), 150)",0.640902,0.799456,0.646783,0.573949
2,BR(DPH),0.275001,0.620768,0.322488,0.298724


### 